In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from nltk import word_tokenize, pos_tag
from tqdm import tqdm, tqdm_notebook

### Load data

In [3]:
path = '../data/'
fname = '{}hatebase_slurs.txt'.format(path)
#fname = '{}hatebase+zeerak_exclude_slurs.txt'.format(path)
slurs = pd.read_csv(fname, header=None)[0].values

In [17]:
#path = '../data/davidson/'
path = '../data/zeerak_naacl/'
# path = '../data/wiki_talk/'
# debug = pd.read_csv('{}debug.csv'.format(path), encoding='utf-8')
train = pd.read_csv('{}train.csv'.format(path), encoding='utf-8')
dev = pd.read_csv('{}dev.csv'.format(path), encoding='utf-8')
test = pd.read_csv('{}test.csv'.format(path), encoding='utf-8')

In [19]:
df = pd.read_csv('{}zeerak_naacl_tweets.csv'.format(path), encoding='utf-8')

In [26]:
sum([v for n, v in Counter(df['user_screen_name']).most_common()[:5]])

11565

In [29]:
len(Counter(df['user_screen_name']))

1771

In [28]:
11_565 /15815

0.7312677837496048

In [27]:
len(df)

15815

In [11]:
sum([*map(bool, df['offensive_language'])])

21308

In [10]:
[*map(bool, df['offensive_language'])][:5]

[False, True, True, True, True]

In [16]:
sum(df['class'] == 2)

4163

In [22]:
df.head()

,tweet_id,created_at,in_reply_to_status_id,lang,retweet_count,user_id,user_screen_name,user_name,text,tags,label
0,552247283438587905,Mon Jan 05 23:36:27 +0000 2015,5.522464236981576e+17,en,0.0,2.941146e+09,VileIslam,Levi Stein,"@ummayman90 @wood_brwood333 For example, in Me...",[],racism
1,552260022130864128,Tue Jan 06 00:27:04 +0000 2015,5.5225810437760614e+17,en,0.0,2.941146e+09,VileIslam,Levi Stein,"@ummayman90 Again, your entire concept of god ...","[{'text': 'Islam', 'indices': [127, 133]}]",racism
2,551769698917167104,Sun Jan 04 15:58:42 +0000 2015,NaN,en,0.0,2.941146e+09,VileIslam,Levi Stein,"@anjemchoudary Your prophet was a rapist, murd...",[],racism
3,551984700764332032,Mon Jan 05 06:13:02 +0000 2015,NaN,en,47.0,2.941146e+09,VileIslam,Levi Stein,RT @DilanaKurdi: Yazidi children who are taken...,"[{'text': 'Sinjar', 'indices': [96, 103]}, {'t...",racism
4,552133306553073665,Mon Jan 05 16:03:33 +0000 2015,NaN,en,0.0,2.941146e+09,VileIslam,Levi Stein,@anjemchoudary Accepting the idea that Allah o...,[],racism


In [5]:
def safe_join(word_list):
    joined = ''
    last_word = None
    for i, w in enumerate(word_list):
        if last_word == '<' or w == '>' or i == 0:
            joined += w
        else:
            joined += ' ' + w
        last_word = w
    return joined

def pos_replace(tweet):
    """Repalce slurs with their POS tags."""
    tokenized_tweet = word_tokenize(tweet)
    pos_tweet = pos_tag(tokenized_tweet)
    pos_replaced = [t if t not in slurs else pos for (t, pos) in pos_tweet]
    return safe_join(pos_replaced)

def unk_replace(tweet):
    """Replace slurs with <UNK>."""
    tokenized_tweet = word_tokenize(tweet)
    unk_replaced = [t if t not in slurs else '<UNK>' for t in tokenized_tweet]
    return safe_join(unk_replaced)

def slur_remove(tweet):
    """Remove slurs from the sentence."""
    tokenized_tweet = word_tokenize(tweet)
    slur_removed = [t for t in tokenized_tweet if t not in slurs ]
    return safe_join(slur_removed)
    

In [6]:
all_datasets = {'train': train, 'test': test, 'dev': dev}
new_cols = ['tweet_unk_slur', 'tweet_no_slur', 'tweet_pos_slur']

for k, d in all_datasets.items():
    tqdm.write('Processing {}, length: {}'.format(k, len(d)))
    tweets = d['tweet'].values
    proc_tweets = []
    for c in new_cols:
        d[c] = ''
    for i, t in enumerate(tqdm_notebook(tweets)):
        proc_tweets.append([unk_replace(t), slur_remove(t), pos_replace(t)])
    d[new_cols] = proc_tweets

Processing train, length: 19856


A Jupyter Widget

KeyboardInterrupt: 

In [7]:
train.to_csv('{}train.csv'.format(path), index=None, encoding='utf-8')
dev.to_csv('{}dev.csv'.format(path), index=None, encoding='utf-8')
test.to_csv('{}test.csv'.format(path), index=None, encoding='utf-8')